In [59]:
import json
from transformers import pipeline
from tqdm.auto import tqdm

In [60]:
def get_prompts(filename):
    with open(filename) as f:
        prompts = []
        for line in f:
            prompts.append(json.loads(line))
    return prompts

In [61]:
def get_generator(model_name):
    generator = pipeline("text-generation", model=model_name, device_map="auto")
    return generator

In [62]:
def generate_suggestions(generator, prompts, key = "prompt", max_new_length = 128, num_suggestions=10):
    suggestions = []
    for prompt in tqdm(prompts):
        updated_prompt = prompt.copy()
        suggestion = generator(prompt[key], max_new_tokens=max_new_length, num_return_sequences=num_suggestions)
        updated_prompt["suggestions"] = suggestion
        suggestions.append(updated_prompt)
    return suggestions

In [63]:
def write_suggestions(suggestions, filename):
    with open(filename, "w") as f:
        for suggestion in suggestions:
            f.write(json.dumps(suggestion) + "\n")

In [64]:
benchmark_root = "../Benchmarks/"
benchmark_file = "HumanEval_python.jsonl"
benchmark_path = benchmark_root + benchmark_file


In [65]:
model_name = "Salesforce/codegen-350M-mono"
max_new_length = 128
num_suggestions = 10

In [75]:
suggestion_root = "./Suggestions/"
suggestion_file = benchmark_file.split(".")[0] + "_" + model_name.split("/")[-1] +"_"+str(max_new_length)+"_"+str(num_suggestions)+ ".jsonl"
suggestion_path = suggestion_root + suggestion_file

In [67]:
prompts = get_prompts(benchmark_path)
print(prompts[0])

{'task_id': 'HumanEval/0', 'prompt': 'from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n', 'entry_point': 'has_close_elements', 'canonical_solution': '    for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False\n', 'test': "\n\nMETADATA = {\n    'author': 'jt',\n    'dataset': 'test'\n}\n\n\ndef check(candidate):\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False\n    assert candidate([1.0, 2.0, 5.9, 4.0

In [68]:
generator = get_generator(model_name)

In [69]:
suggestions = generate_suggestions(generator, prompts, key = "prompt", max_new_length=max_new_length, num_suggestions=num_suggestions)

100%|██████████| 164/164 [1:20:29<00:00, 29.45s/it]


In [76]:
write_suggestions(suggestions, suggestion_path)

In [71]:
py_model_list = ["Salesforce/codegen-350M-mono", "Salesforce/codegen-2B-mono","Salesforce/codegen-6B-mono", "codeparrot/codeparrot-small","codeparrot/codeparrot"]

In [72]:
multi_model_list = [ "Salesforce/codegen-350M-multi",  "Salesforce/codegen-2B-multi","Salesforce/codegen-6B-multi", "codeparrot/codeparrot-small-multi", "NinedayWang/PolyCoder-160M","NinedayWang/PolyCoder-0.4B","NinedayWang/PolyCoder-2.7B", "facebook/incoder-1B", "facebook/incoder-6B"]

In [73]:
combined_model_list = py_model_list + multi_model_list
len(combined_model_list)

14

In [78]:
for model in combined_model_list:
    print(model)
    generator = get_generator(model)
    

Salesforce/codegen-350M-mono
Salesforce/codegen-2B-mono
Salesforce/codegen-6B-mono
codeparrot/codeparrot-small
codeparrot/codeparrot
Salesforce/codegen-350M-multi
Salesforce/codegen-2B-multi
Salesforce/codegen-6B-multi
codeparrot/codeparrot-small-multi
NinedayWang/PolyCoder-160M
NinedayWang/PolyCoder-0.4B
NinedayWang/PolyCoder-2.7B
facebook/incoder-1B
facebook/incoder-6B
